## Monte Carlo Geometry Processing

Monte Carlo PDE estimator computes the solution to a partial differential equation at some given point with the Walk on Spheres(WoS) algorithm.


### Laplacian Equation


For a Laplacian equation given by 
$$ \Delta u = 0 \ \ \ on \ \ \Omega$$
$$ u = g \ \ \ on \ \ \partial\Omega$$
The WoS algorithm is done iteratively, for an iteration k:
1. At $x_k$, find the largest sphere $S(x_k)$ centered at $x_k$ inscribed in the mesh
2. Pick a random point $x_{k+1}$ on $S(x_k)$

Repeat step 1 and 2 until $x_k$ is close to the mesh, i.e. $d(x_k, V, F) < \epsilon$  
Give an estimate for $u(x_0)$ as $\hat{u}(x_0)$ = $g(\bar{x_k})$, where $\bar{x_k}$ is the closest point on mesh to $x_k$

In libigl, this estimator can be used as follows:

Where $B$ is a function $\mathbb{R}^3 \rightarrow \mathbb{R}$ that can be evaluated on points on mesh. $P$ is a set of points (not necessarily part of any mesh).

In [2]:
# bunny_laplacian, cactus_laplacian

Figure 1. solved laplacian with the boundary condition $g(x) = \|x\|_2$ 

### Poisson Equation

For a Poisson equation given by
$$\Delta u = f \ \ \ on \ \ \Omega$$
$$u = g \ \ \ on\ \  \partial\Omega$$

The WoS algorithm is similar to the one for Laplacian, except for an iteration k, it has an extra step:
3. Pick a random point $y_k$ in the largest ball $B(x_k)$ centered at $x_k$ inscribed in the mesh

And the estimate for $u(x_0)$ is $\hat{u}(x_0) = g(\bar{x_0}) + \Sigma_{i=1}^k |B(x_i)|f(y_i)G(x_i, y_i)$

In libigl, this estimator can be used as follows:

Where $f$ is ...

#### Importance Sampling

For a specific type of source function $f_z = c \delta_z$, where c is a constant and $\delta_z$ is the Dirac delta centered at a point $z\in \Omega$  
TODO: paper section 4.2.2

In [ ]:
# without importance, with importance

Figure 2. solved poisson with .....